In [212]:
import math
import torch

import gpytorch

from matplotlib import pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np

- Test fix of NN kernel as is has shown to be usefull in other works
- heteroscedastic noise: 
- MTGP: shared covariance function and shared paraemtrisation or not?

Environment: Use .conda-deepsensor now because we don't need xarray

## Load data

In [136]:
data = pd.read_csv("data/merged.csv")

## Isotopic case: 

In [137]:
# Select rows where we have data for both
mb_df = data[~data.amb_nan & ~data.gmb_nan]

In [147]:
# 4518 rows x 2 columns (x and y)
mb_train_x_unnorm = torch.tensor(np.array(mb_df[["x", "y"]])).type(torch.float32)
normalising_constant = mb_train_x_unnorm.max()
print("Normalising constant is", normalising_constant.item())
# Normalise to [-1, 1] by dividing by the normalising constant
mb_train_x = mb_train_x_unnorm / normalising_constant

# 4518 rows x 2 columns (amb and gmb)
mb_train_y = torch.tensor(np.array(mb_df[["amb", "gmb"]])).type(torch.float32)

Normalising constant is 2550000.0


## Check distribution

- need for normalisation? Not really.

In [148]:
print(mb_train_y.std(dim = 0))
# std of amb is much larger than gmb because gmb is much smoother
print(mb_train_y.mean(dim = 0))
# mean is close enough to 0

tensor([0.1534, 0.0705])
tensor([-0.0082, -0.0080])


In [149]:
import plotly.express as px

fig = px.histogram(mb_train_y[:, 0])
fig.update_layout(title_text = 'Histogram of amb')

fig.update_layout(template = "plotly_white")
fig.update_layout(font_family = "Lato")

fig.show()

In [150]:
fig = px.histogram(mb_train_y[:, 1])
fig.update_layout(title_text = 'Histogram of gmb')

fig.update_layout(template = "plotly_white")
fig.update_layout(font_family = "Lato")

fig.show()

In [151]:
class ZM_RBF_MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ZM_RBF_MultitaskGPModel, self).__init__(train_x, train_y, likelihood)

        # Wrap mean in multitask mean
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ZeroMean(), num_tasks = 2
        )

        # Already does scaling for us
        # Full rank: number of tasks
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims = 2), num_tasks = 2, rank = 2
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

In [152]:
mb_likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks = 2, rank = 0)
mb_model = ZM_RBF_MultitaskGPModel(mb_train_x, mb_train_y, mb_likelihood)

In [162]:
# this is for running the notebook in our testing framework
import os
# Continuous Integration (CI) environment
smoke_test = ('CI' in os.environ)
training_iterations = 2 if smoke_test else 100

# Find optimal model hyperparameters
mb_model.train()
mb_likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(mb_model.parameters(), lr = 0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(mb_likelihood, mb_model)

for i in range(training_iterations):
    optimizer.zero_grad()
    output = mb_model(mb_train_x)
    loss = - mll(output, mb_train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
    optimizer.step()

Iter 1/100 - Loss: 0.903
Iter 2/100 - Loss: 0.863
Iter 3/100 - Loss: 0.822
Iter 4/100 - Loss: 0.781
Iter 5/100 - Loss: 0.740
Iter 6/100 - Loss: 0.697
Iter 7/100 - Loss: 0.655
Iter 8/100 - Loss: 0.612
Iter 9/100 - Loss: 0.568
Iter 10/100 - Loss: 0.524
Iter 11/100 - Loss: 0.480
Iter 12/100 - Loss: 0.436
Iter 13/100 - Loss: 0.391
Iter 14/100 - Loss: 0.346
Iter 15/100 - Loss: 0.300
Iter 16/100 - Loss: 0.255
Iter 17/100 - Loss: 0.209
Iter 18/100 - Loss: 0.163
Iter 19/100 - Loss: 0.117
Iter 20/100 - Loss: 0.071
Iter 21/100 - Loss: 0.025
Iter 22/100 - Loss: -0.020
Iter 23/100 - Loss: -0.066
Iter 24/100 - Loss: -0.112
Iter 25/100 - Loss: -0.157
Iter 26/100 - Loss: -0.202
Iter 27/100 - Loss: -0.247
Iter 28/100 - Loss: -0.291
Iter 29/100 - Loss: -0.335
Iter 30/100 - Loss: -0.378
Iter 31/100 - Loss: -0.420
Iter 32/100 - Loss: -0.462
Iter 33/100 - Loss: -0.503
Iter 34/100 - Loss: -0.544
Iter 35/100 - Loss: -0.583
Iter 36/100 - Loss: -0.622
Iter 37/100 - Loss: -0.660
Iter 38/100 - Loss: -0.698
Iter

In [163]:
current_model = mb_model

print("TASK COVARIANCE:")
B = current_model.covar_module.task_covar_module.covar_factor.data.matmul(current_model.covar_module.task_covar_module.covar_factor.data.transpose(-1, -2)) + torch.diag_embed(torch.nn.functional.softplus(current_model.covar_module.task_covar_module.raw_var.data))
print(B)

print()
print("TASK correlation coefficient:")
corr_coeff = B[0, 1] / torch.sqrt(B[0, 0] * B[1, 1])
print(corr_coeff)

print()
print("NOISE (CO)VARIANCE:")
D = torch.diag_embed(current_model.likelihood.task_noises.detach()) + (current_model.likelihood.noise.detach() * torch.eye(2))
print(D)

print()
print("RBF base kernel lengthscale:")
print(torch.nn.functional.softplus(current_model.covar_module.data_covar_module.raw_lengthscale).detach())

TASK COVARIANCE:
tensor([[8.7528, 0.2146],
        [0.2146, 0.0089]])

TASK correlation coefficient:
tensor(0.7673)

NOISE (CO)VARIANCE:
tensor([[0.0063, 0.0000],
        [0.0000, 0.0004]])

RBF base kernel lengthscale:
tensor([[0.0911, 0.1149]])


Observations:
- Lengthscales (ARD) always get initialised as 0.6931 each.
- Noise always gets initialised as 1.3865 (diagonal).
- Tasks are initially uncorrelated/correlated/anti-correleated.

TASK COVARIANCE:
tensor([[1.7223, 0.9812],
        [0.9812, 8.1079]])

TASK correlation coefficient:
tensor(0.2626)

In [164]:
# Set into eval mode
mb_model.eval()
mb_likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # Same data as training?!
    predictions = mb_likelihood(mb_model(mb_train_x))

/home/kim/new_massbalance/mass_balance_ConvNP/.conda-deepsensor/lib/python3.11/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning:

The input matches the stored training data. Did you forget to call model.train()?



# ToDO

- function to go from tensor data (dense table) back to (sparse) image data.
- don't take full data every time in training
- normalise X data 

In [165]:
def convert_to_sparse(array_populated_indices, predictions, full_coordinates_df):
    """_summary_

    Args:
        array_populated_indices (np.array of length (S,)): array of indices that are populated
        predictions (object): MultitaskMultivariateNormal(mean shape: torch.Size([S, 2])) object with mean and variance
        full_coordinates_df (pd.Dataframe): DataFrame containing all coordinates of the grid, with min a column called "x" and a column called "y". Must contain index too.

    Returns:
        _type_: _description_
    """
    indexed_means = pd.DataFrame(np.concatenate([array_populated_indices.reshape(-1, 1), predictions.mean.numpy()], axis = 1), columns =["index", "amb", "gmb"])

    indexed_var = pd.DataFrame(np.concatenate([array_populated_indices.reshape(-1, 1), predictions.variance.numpy()], axis = 1), columns =["index", "amb", "gmb"])

    # make index a column so we can use it for a merge
    full_coordinates_df["index"] = full_coordinates_df.index

    # Outer join to keep all rows
    full_means = full_coordinates_df[['index', 'x', 'y']].merge(indexed_means, on = "index", how = "outer")

    full_var = full_coordinates_df[['index', 'x', 'y']].merge(indexed_var, on = "index", how = "outer")

    amb_pred_mean = full_means["amb"].to_numpy().reshape(97, 117)
    gmb_pred_mean = full_means["gmb"].to_numpy().reshape(97, 117)
    amb_pred_var = full_var["amb"].to_numpy().reshape(97, 117)
    gmb_pred_var = full_var["gmb"].to_numpy().reshape(97, 117)

    return amb_pred_mean, gmb_pred_mean, amb_pred_var, gmb_pred_var

In [166]:
amb_pred_mean, gmb_pred_mean, amb_pred_var, gmb_pred_var = convert_to_sparse(np.array(mb_df.index), predictions, data)

In [194]:
# Min is always of higher magnitude than max
cscale_min = np.min([np.nanmin(amb_pred_mean), np.nanmin(gmb_pred_mean)])

In [230]:
# Heatmap requires a colorscale with a list of tuples
# This colorscale is very sensitive to values close to 0 so we can see something.
nasa_cscale = [(0, "#371229"), (0.15, "#B6222B"), (0.3, "#FF8E35"), (0.475, "#FCDC79"),(0.5, "white"), (0.525, "#D2EEFF"), (0.75,"#7DC0E8"), (1,"#2D9EB9")]

fig = go.Figure(go.Heatmap(
    # mean or sum
    z = amb_pred_mean,
        colorscale = nasa_cscale,
    zmid = 0,
    zmin = cscale_min,
    zmax = -cscale_min
    ))

fig.update_layout(
    title = "ALTIMETRY mass balance (2003-2018 yearly average)",
)

fig.update_traces(colorbar_orientation = "v")
fig.update_traces(colorbar_title = "meters w.e.")
fig.update_traces(colorbar_len = 0.7)

fig.update_layout(
    height = 600,
    width = 600,
    yaxis_scaleanchor = "x",
)

fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)

fig.update_layout(template = "plotly_white")
fig.update_layout(font_family = "Lato")

fig.update_layout(
    {"paper_bgcolor": "rgba(0, 0, 0, 0)", "plot_bgcolor": "rgba(0, 0, 0, 0)"})

fig.show()


# Plot both at the same time

In [228]:
# install make_subplots
fig = make_subplots(1, 2)

fig.update_layout(
    height = 600,
    width = 1300,
)

fig.add_trace(go.Heatmap(
    # mean or sum
    z = amb_pred_mean,
    colorscale = nasa_cscale,
    zmid = 0,
    zmin = cscale_min,
    zmax = -cscale_min
    ), 1, 1)

fig.add_trace(go.Heatmap(
    # mean or sum
    z = gmb_pred_mean,
    colorscale = nasa_cscale,
    zmid = 0,
    zmin = cscale_min,
    zmax = -cscale_min,
    ), 1, 2)

fig.update_layout(
    {"paper_bgcolor": "rgba(0, 0, 0, 0)", "plot_bgcolor": "rgba(0, 0, 0, 0)"})

fig.update_layout(
    title = "ALTIMETRY (left) vs. GRAVITY (right) mass balance (2003-2018 yearly average)",
)

fig.update_yaxes(visible = False, showticklabels = False)
fig.update_xaxes(visible = False, showticklabels = False)

fig.show()

# Variance

We observe data everywhere so there is none

In [229]:
# install make_subplots
fig = make_subplots(1, 2)

fig.update_layout(
    height = 600,
    width = 1300,
)

fig.add_trace(go.Heatmap(
    # mean or sum
    z = amb_pred_var,
    colorscale = nasa_cscale,
    zmid = 0,
    zmin = cscale_min,
    zmax = -cscale_min
    ), 1, 1)

fig.add_trace(go.Heatmap(
    # mean or sum
    z = gmb_pred_var,
    colorscale = nasa_cscale,
    zmid = 0,
    zmin = cscale_min,
    zmax = -cscale_min,
    ), 1, 2)

fig.update_layout(
    {"paper_bgcolor": "rgba(0, 0, 0, 0)", "plot_bgcolor": "rgba(0, 0, 0, 0)"})

fig.update_layout(
    title = "ALTIMETRY (left) vs. GRAVITY (right) mass balance (2003-2018 yearly average)",
)

fig.update_yaxes(visible = False, showticklabels = False)
fig.update_xaxes(visible = False, showticklabels = False)

fig.show()